*Developed using conda python 3.4 environment with netCDF4 package installed*

In [1]:
import urllib.request
import os.path
import numpy
import struct
import netCDF4

In [2]:
def fetch_file_from_web(url, filename=None):
    """Download file from url and optionally names it as filename"""
    if filename is None: local_file = os.path.split(url)[-1]
    else: local_file = filename
    if not os.path.isfile(local_file):
        print('\rDownload file %s ...'%local_file, end='')
        urllib.request.urlretrieve(url, local_file)
        print('\rDone.', end='')

In [3]:
def convert_ETOPO5_to_netcdf(etopo5_file='ETOPO5.DAT', netcdf_file='ETOPO5.nc'):
    """Creates a netcdf containing the ETOPO5 dataset.
    Will download ETOPO5.DAT if it is not present."""
    if not os.path.isfile(etopo5_file):
        raise Exception('Download ETOPO5.DAT from https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO5/TOPO/ETOPO5/ETOPO5.DAT')
    # The documentation at http://www.ngdc.noaa.gov/mgg/global/etopo5.HTML states that
    #   "The data start at the North Pole (90 deg N, 0 deg 0' E) and are arranged in bands of
    #    360 degrees x 12 points/degree = 4320 values (8640 bytes) ranging eastward from
    #    0 deg 0' East longitude to 359 deg 55' East longitude (since it represents the North
    #    Pole, all possible longitudes still refer to a single point, thus the first band has
    #    4320 identical values of -4290 m). The 8641st starts the latitude band for 89 deg 55' N,
    #    and so on. There is NO record for the South Pole (elevation 2810 m.)"
    # Therefore, we must add a row for the south pole.
    nj,ni = 2161,4320
    H = numpy.zeros((nj,ni))
    with open('ETOPO5.DAT','rb') as fid:
        for j in range(nj-1):
            print('\rReading row %i/%i ...'%(j+1,nj-1), end='')
            for i in range(ni):
                b = fid.read(2)
                H[j,i] = struct.unpack('>h',b)[0]
    print('\rFlipping data...', end='')
    H[-1,:] = 2810. # For the missing south pole data
    H = H[::-1,:] # South to north instead of north to south
    x = numpy.arange(ni) * (360. / ni)
    y = numpy.arange(nj) * (180. / (nj-1)) - 90.
    print('\rCreating netcdf file ...', end='')
    # Create netcdf file (http://unidata.github.io/netcdf4-python/#netCDF4.Dataset.__init__)
    rg = netCDF4.Dataset(netcdf_file, mode='w', format='NETCDF3_64BIT')
    # Create dimensions (http://unidata.github.io/netcdf4-python/#netCDF4.Dataset.createDimension)
    rg.createDimension('ni', ni)
    rg.createDimension('nj', nj)
    # Create variables (http://unidata.github.io/netcdf4-python/#netCDF4.Dataset.createVariable)
    ncvar_x = rg.createVariable('lon', 'f4', ('ni',))
    ncvar_y = rg.createVariable('lat', 'f4', ('nj',))
    ncvar_e = rg.createVariable('elevation', 'f4', ('nj','ni',))
    # Attributes
    rg.title = 'ETOPO5 5-minute gridded elevation data'
    rg.url = 'http://www.ngdc.noaa.gov/mgg/global/etopo5.HTML'
    rg.citation = ('Data Announcement 88-MGG-02, Digital relief of the Surface of the Earth.'
    +' NOAA, National Geophysical Data Center, Boulder, Colorado, 1988.')
    rg.description = ('ETOPO5 was generated from a digital data base of land and sea-floor elevations'
    +' on a 5-minute latitude/longitude grid. The resolution of the gridded data varies from true 5-minute'
    +' for the ocean floors, the USA., Europe, Japan,and Australia to 1 degree in data-deficient parts of'
    +' Asia, South America, northern Canada, and Africa. Data sources are as follows: Ocean Areas: US Naval'
    +' Oceanographic Office; USA., W. Europe, Japan/Korea: US Defense Mapping Agency; Australia: Bureau of'
    +' Mineral Resources, Australia; New Zealand: Department of Industrial and Scientific Research, New'
    +' Zealand; balance of world land masses: US Navy Fleet Numerical Oceanographic Center. These various'
    +' data bases were originally assembled in 1988 into the worldwide 5-minute grid by Margo Edwards, then'
    +' at Washington University, St. Louis, MO.')
    ncvar_x.description = 'Longitude of grid nodes'
    ncvar_x.units = 'degrees east'
    ncvar_y.description = 'Latitude of grid nodes'
    ncvar_y.units = 'degrees north'
    ncvar_e.desciption = 'Elevation of land and sea-floor given as height at nodes'
    ncvar_e.units = 'm'
    print('\rWriting to netcdf file ...', end='')
    ncvar_x[:] = x
    ncvar_y[:] = y
    ncvar_e[:,:] = H
    rg.close()
    print('\rDone.', end='')

In [4]:
fetch_file_from_web('https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO5/TOPO/ETOPO5/ETOPO5.DAT')
convert_ETOPO5_to_netcdf()

Done.